<a href="https://colab.research.google.com/github/Abhiraj36/HumanEmotionsDetection/blob/main/Emotion_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/Abhiraj36/HumanEmotionsDetection.git
%cd HumanEmotionsDetection
from tensorflow.keras.models import load_model

lenet_model = load_model('model_full.keras', compile=False)  # or use the path inside the repo if nested
lenet_model.summary()  # optional: to see the model structure


Cloning into 'HumanEmotionsDetection'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 53 (delta 17), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 4.70 MiB | 12.53 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/HumanEmotionsDetection/HumanEmotionsDetection


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_9 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 109, 109, 68)   │        19,652 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 109, 109, 68)   │           272 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 54, 54, 68)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 54, 54, 68)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 68)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │         8,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,999 (152.34 KB)

 Trainable params: 38,415 (150.06 KB)

 Non-trainable params: 584 (2.28 KB)

In [ ]:
import kagglehub
from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json




# Download both datasets
facial_emotion_path = kagglehub.dataset_download("himanshuydv11/facial-emotion-dataset")
human_emotion_path = kagglehub.dataset_download("muhammadhananasghar/human-emotions-datasethes")

# ✅ Copy to /content instead of /kaggle/input
!cp -r "{facial_emotion_path}" /content/facial-emotion-dataset
!cp -r "{human_emotion_path}" /content/human-emotions-datasethes

# Update paths
facial_dataset_path = "/content/facial-emotion-dataset/facial_emotion_dataset/dataset"
human_dataset_path = "/content/human-emotions-datasethes/Human_Emotion_Dataset"

# Merge selected classes
selected_classes = ['Sad', 'Angry', 'Happy']

for cls in selected_classes:
    src = os.path.join(facial_dataset_path, cls)
    dst = os.path.join(human_dataset_path, cls)

    if not os.path.exists(src):
        print(f"❌ Skipping '{cls}' - not found in source dataset")
        continue

    os.makedirs(dst, exist_ok=True)

    for file_name in os.listdir(src):
        src_file = os.path.join(src, file_name)
        dst_file = os.path.join(dst, file_name)
        if os.path.isfile(src_file):
            shutil.copy(src_file, dst_file)

print("✅ Merged selected classes successfully!")





Saving kaggle.json to kaggle (3).json


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

validation_dir = "/content/human-emotions-datasethes/Human_Emotion_Dataset"  # or wherever your val set is

val_datagen = ImageDataGenerator(rescale=1./255)

validation_dataset = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 0 images belonging to 1 classes.


In [12]:
lenet_model.evaluate(validation_dataset)

NameError: name 'validation_dataset' is not defined